In [2]:
#Import Statements
import deepchem as dc
from rdkit import Chem
from rdkit.Chem import Draw
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, losses

import pandas as pd

from matplotlib import pyplot as plt

import numpy as np

from sklearn import metrics
from sklearn.metrics import f1_score

import keras
from keras import layers

In [3]:
# #GraphConv featuriser - splitting the training dataset into smaller datasets
# tasks, datasets, transformers = dc.molnet.load_tox21(
#     featurizer='GraphConv', 
#     save_dir=r'C:\Users\ym20201\Documents\Datasets',
#     data_dir=r'C:\Users\ym20201\Documents\Datasets')
# splitter = dc.splits.RandomSplitter()

# train_data, valid_data,test_data = splitter.train_valid_test_split(
#     datasets[0], 
#     frac_train = 0.8, frac_valid = 0.08, frac_test = 0.12)


# # model = dc.models.GraphConvModel(n_tasks=12, mode='regression', dropout=0.2)

In [4]:
#GraphConv featuriser - splitting the training dataset into smaller datasets
tasks, datasets, transformers = dc.molnet.load_tox21(
    save_dir=r'C:\Users\ym20201\Documents\Datasets',
    data_dir=r'C:\Users\ym20201\Documents\Datasets')

featurizer = dc.feat.ConvMolFeaturizer()
splitter = dc.splits.RandomSplitter()

train_data, valid_data,test_data = splitter.train_valid_test_split(
    datasets[0], 
    frac_train = 0.8, frac_valid = 0.08, frac_test = 0.12)

#Featurisation using ConvMolFeaturizer
train_feat = featurizer.featurize(train_data.ids)
valid_feat = featurizer.featurize(valid_data.ids)
test_feat = featurizer.featurize(test_data.ids)

# model = dc.models.GraphConvModel(n_tasks=12, mode='regression', dropout=0.2)

[15:09:33] WARNING: not removing hydrogen atom without neighbors


In [15]:
# #epoch - number of times data is shown to NN
# model.fit(train_data, nb_epoch=10) #training model
train_feat

array([<deepchem.feat.mol_graphs.ConvMol object at 0x000001BC0B3D3A88>,
       ...,
      dtype=object)

In [22]:
# tensor = tf.convert_to_tensor(train_data.ids[0])
# tensor = tf.convert_to_tensor(train_feat)


In [14]:
#Converting numpy array to tensor
train_tensor = []
for i in range(len(train_feat)):
    tensor = tf.convert_to_tensor(train_feat[i])
    train_tensor.append(tensor)
    
valid_tensor = []
for i in range(len(valid_feat)):
    tensor_v = tf.convert_to_tensor(valid_feat[i])
    valid_tensor.append(tensor_v)
    
test_tensor = []
for i in range(len(test_feat)):
    tensor_t = tf.convert_to_tensor(test_feat[i])
    test_tensor.append(tensor_t)

ValueError: Attempt to convert a value (<deepchem.feat.mol_graphs.ConvMol object at 0x000001BC0B3D3A88>) with an unsupported type (<class 'deepchem.feat.mol_graphs.ConvMol'>) to a Tensor.

In [22]:
# metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
# print("Training set score:", model.evaluate(train_data, [metric], transformers))
# print("Test set score:", model.evaluate(test_data, [metric], transformers))

In [5]:
#Constructing the model
# keras_model = Sequential([
#     layers.Dense(50, activation='relu'), # hidden layer
#     layers.Dense(1) # output layer
# ])

# model = dc.models.KerasModel(keras_model, dc.models.losses.L1Loss())
# model.fit(train_feat, nb_epoch=10)

keras_model = Sequential()
keras_model.add(layers.Conv2D(filters=32,
                          kernel_size=3,
                          activation='relu',
                          dilation_rate=2,
                             input_shape=(80, 80, 1)))
keras_model.add(layers.MaxPooling2D(pool_size=(2,2)))
#dropout to avoid overfitting
keras_model.add(layers.Dense(50, activation='relu'))
keras_model.add(layers.Dense(1))

keras_model.build()

In [6]:
keras_model.compile(optimizer='adam', loss='mean_squared_error')
keras_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 76, 76, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 38, 38, 32)       0         
 )                                                               
                                                                 
 dense (Dense)               (None, 38, 38, 50)        1650      
                                                                 
 dense_1 (Dense)             (None, 38, 38, 1)         51        
                                                                 
Total params: 2,021
Trainable params: 2,021
Non-trainable params: 0
_________________________________________________________________


In [45]:
# tf.convert_to_tensor(train_feat[0])


In [13]:
keras_model.fit(x=train_feat,
                y=train_feat,
                epochs=10,
                batch_size=512,
                shuffle=True,
               validation_data=(valid_feat, valid_feat))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type ConvMol).

In [12]:
#Autoencoder